In [1]:
import os
import glob
import json

In [2]:
model_names = [
    "distilroberta-base"
]

raw_directories = [d for d in os.listdir(os.getcwd()) if os.path.isdir(d)]
result_directories = []
for d in raw_directories:
    for m in model_names:
        if m in d:
            result_directories.append(d)
            break
            
result_directories.sort()

In [5]:
result_directories

['distilroberta-base-128-128',
 'distilroberta-base-32-128',
 'distilroberta-base-64-128',
 'distilroberta-base-8-128',
 'distilroberta-base-8-512']

In [3]:
def load_gpu_util(dlprof_summary_file):
    with open(dlprof_summary_file) as json_file:
        summary = json.load(json_file)
    gpu_util_raw = summary["Summary Report"]
    gpu_util = {
        "sm_util": int(100 - gpu_util_raw["GPU Idle %"][0]),
        "tc_util": int(gpu_util_raw["Tensor Core Kernel Utilization %"][0])
    }
    return gpu_util

def parse_deviceQuery(gpu_hw_file):
    lines = [line.rstrip("\n") for line in open(gpu_hw_file)]
    for l in lines:
        if 'Device 0: "' in l:
            gpu_name = l.split('Device 0: "')[-1]
            gpu_name = gpu_name.replace('"', "").strip()
    gpu_data = {
        "gpu_name": gpu_name
    }
    return gpu_data

def parse_pl_timings(pl_profile_file):
    lines = [line.rstrip("\n") for line in open(pl_profile_file)]
    mean_timings = {}
    for l in lines[7:]:
        if "|" in l:
            l = l.split("|")
            l = [i.strip() for i in l]
            mean_timings[l[0]] = float(l[1])
    return mean_timings

for d in result_directories:
    # gather info
    _d = d.split("-")
    seq_len = int(_d[-1])
    batch_size = int(_d[-2])
    model_name = "-".join(_d[:-2])
    gpu_data = parse_deviceQuery(d+"/gpu_hw_info.txt")
    gpu_name = gpu_data["gpu_name"]
    # parse results
    gpu_util = load_gpu_util(d+"/dlprof_summary.json")
    step_time = parse_pl_timings(d+"/pl_profile.txt")["optimizer_step"]
    fps = int(batch_size / step_time)
    print(step_time, fps)

0.16113 794
0.05072 630
0.087522 731
0.024786 322
0.067987 117
